In [1]:
#general libraries
import pandas as pd
import numpy as np
import copy
import wandb
#project code
from FNN import FNN
from gradient_descent import *
from ActivFunctions import  *
from LossFunctions import *
from SuppFunctions import *
from TestingFunctions import *
from SweepFunctions import *

In [ ]:
api_key = ""#specify your api_key, but never share it
entity = "DL_project_Group_70"
project = "test"

In [3]:
wandb.login(api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Normal\_netrc
wandb: Currently logged in as: s253711 (DL_project_Group_70) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
sweep_configuration = {
    "name": "sweepdemo",
    "method": "bayes",
    "metric": {"goal": "minimize", "name": "val_loss"},
    "parameters": {
        "dataset": {"values": ["MNIST"]},#"MNIST","CIFAR"
        "epochs": {"values": [20,25,30]},
        "num_hidden_layers": {"values": [3, 4, 5]},
        "num_hidden_units": {"values": [ 64, 128, 256]},
        "optimizer": {"values": ["sgd","sgd_momentum","rmsprop","nag","adam"]},
        "learning_rate": {"min": 0.0001, "max": 0.01},
        "grad_clip": {"min": 0.5, "max": 5.0},
        "momentum": {"min": 0.8, "max": 0.95},#sgd_momentum,nag
        "beta": {"min": 0.9, "max": 0.99},#rmsprop
        "beta1": {"min": 0.8, "max": 0.95},#adam
        "beta2": {"min": 0.96, "max": 0.99},#adam
        "batch_size": {"values": [ 64, 128, 256]},
        "l_coeff": {"min": 1e-5, "max": 1e-3},
        "l_method": {"values": ["none","l1", "l2"]},
        "weights_init": {"values": ["RandomUni", "RandomNor", "XavUni", "XavNor", "HeUni", "HeNor"]},
        "activation_hidden": {"values": ["identity", "sigmoid", "tanh", "relu", "leaky_relu"]},
        "activation_output": {"values": ["identity", "sigmoid", "tanh", "relu", "leaky_relu", "softmax"]},
        "loss_function": {"values": ["MeanSquaredError", "CrossEntropy", "SoftmaxCrossEntropy"]},
    },
}

In [5]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project = project)

Create sweep with ID: lsje5vqr
Sweep URL: https://wandb.ai/DL_project_Group_70/test/sweeps/lsje5vqr


In [6]:
wandb.agent(sweep_id, function=main,count=1)

wandb: Agent Starting Run: dpzx3lpa with config:
wandb: 	activation_hidden: relu
wandb: 	activation_output: softmax
wandb: 	batch_size: 64
wandb: 	beta: 0.9249506220299388
wandb: 	beta1: 0.841237268198561
wandb: 	beta2: 0.9632296063453496
wandb: 	dataset: MNIST
wandb: 	epochs: 20
wandb: 	grad_clip: 0.5860052515903349
wandb: 	l_coeff: 0.0002457005633966421
wandb: 	l_method: l2
wandb: 	learning_rate: 0.005704586519469704
wandb: 	loss_function: CrossEntropy
wandb: 	momentum: 0.9378796294760948
wandb: 	num_hidden_layers: 5
wandb: 	num_hidden_units: 128
wandb: 	optimizer: adam
wandb: 	weights_init: XavNor


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_acc,▂▆▅▅█▁▇▃█▆▇▆█▄█▇▅█▅▇
train_loss,▇▃▃▆▃▁▄█▄▅▃▂▄▄▃▄▃▃▄▅
val_acc,▂▇▅▅█▁▇▂█▅▆▆█▄▇▇▅▇▅▆
val_loss,▇▃▃▆▄▁▄█▄▅▄▃▄▄▃▄▃▃▄▅
epoch,20
train_acc,0.91765
train_loss,147084.33601
val_acc,0.9166
val_loss,25005.22403
